In [1]:
# imports

import copy
import random as rd
import statistics as st

In [2]:
# unit profiles

class FeralWarpwolf:
    def __init__(self, str_buff=0, mat_buff=0, arm_buff=0):
        # base attributes
        self.base_speed = 6
        self.base_strength = 11
        self.base_mat = 7
        self.base_defense = 14
        self.base_armor = 16
        self.base_fury = 4
        self.base_threshold = 9
        self.base_hp = 30
        self.base_cost = 16
        
        # base boxes
        self.mind1 = 4
        self.mind2 = 3
        self.mind3 = 2
        self.body1 = 5
        self.body2 = 4
        self.body3 = 2
        self.spirit1 = 4
        self.spirit2 = 4
        self.spirit3 = 2
        
        self.mind_disabled = False
        self.body_disabled = False
        self.spirit_disabled = False
        
        # Body: When this aspect is crippled, you will roll one less die on ALL Damage rolls this model makes.
        # Mind: When this aspect is crippled, you will roll one less die on ALL Attack rolls with this model.
        # Spirit: When this aspect is crippled, the model may not be forced, so it will not be able to generate Fury.
        
        # buffs
        self.str_buff = str_buff
        self.mat_buff = mat_buff
        self.arm_buff = arm_buff
        self.prot_plates = 0
        
        # current attributes
        self.speed = self.base_speed
        self.strength = self.base_strength + self.str_buff
        self.mat = self.base_mat + self.mat_buff
        self.defense = self.base_defense
        self.armor = lambda: self.base_armor + self.arm_buff + self.prot_plates
        self.fury = self.base_fury
        self.threshold = self.base_threshold
        self.hp = self.base_hp # this needs to be lambda: boxes
        self.cost = self.base_cost
        
        # weapons
        self.weapons = {
            'Bite': {
                'p+s': 3 + self.strength,
                'rof': 1
            },
            'Claw': {
                'p+s': 4 + self.strength,
                'rof': 2
            }
        }
        
    def protective_plates(self, active=False):
        if active:
            self.prot_plates = 2
        else:
            self.prot_plates = 0
            
        
class TitanGladiator:
    def __init__(self, str_buff=0, mat_buff=0, arm_buff=0):
        # base attributes
        self.base_speed = 4
        self.base_strength = 12
        self.base_mat = 6
        self.base_defense = 10
        self.base_armor = 19
        self.base_fury = 4
        self.base_threshold = 9
        self.base_hp = 30
        self.base_cost = 15
        
        # buffs
        self.str_buff = str_buff
        self.mat_buff = mat_buff
        self.arm_buff = arm_buff
        
        # current attributes
        self.speed = self.base_speed
        self.strength = self.base_strength + self.str_buff
        self.mat = self.base_mat + self.mat_buff
        self.defense = self.base_defense
        self.armor = lambda: self.base_armor + self.arm_buff
        self.fury = self.base_fury
        self.threshold = self.base_threshold
        self.hp = self.base_hp
        self.cost = self.base_cost
        
        self.weapons = {
            'Tusks': {
                'p+s': 3 + self.strength,
                'rof': 1
            },
            'War Gauntlet': {
                'p+s': 4 + self.strength,
                'rof': 2
            }
        }

In [3]:
# functions

def roll_d6(num=1):
    """Rolls a specified number of six-sided dice."""
    
    return rd.randint(num,6)


def melee_attack_roll(model, target, boosted=False):
    """Roll and calculate a melee attack.
    
    model: the attacking model
    target: the defending model
    boosted: is the attack boosted
    """
    
    # roll first die
    first_result = roll_d6()
    
    # roll second die
    if model.mind_disabled:
        second_result = 0
    else:
        second_result = roll_d6()
    
    # roll boost die
    if boosted:
         boost = roll_d6()
    else:
        boost = 0
    
    # calculate mat + dice total
    total_result = model.mat + first_result + second_result + boost
    
    # check for crit or failure
    if first_result == 1 and second_result == 1:
        special = 'Failure!'
    elif first_result==second_result:
        special = 'Crit!'
    
    # calculate hit
    if total_result >= target.defense:
        return 'Hit!'
    else:
        return 'Miss!' 

    
def damage_roll(model, target, powstr, boosted=False):
    """Roll and calculate damage.
    
    powstr: the power+strength of the attacking model
    target: the defending model
    boosted: is the damage boosted
    """
    
    # roll first die
    first_result = roll_d6()
    
    # roll second die
    if model.body_disabled:
        second_result = 0
    else:
        second_result = roll_d6()
    
    # roll boost die
    if boosted:
         boost = roll_d6()
    else:
        boost = 0
    
    # calculate power+strength + dice total
    total_result = powstr + first_result + second_result + boost
    
    # calculate damage
    if total_result > target.armor():
        damage = total_result - target.armor()
    else:
        damage = 0
    
    return damage


def is_dead(unit):
    if unit.hp <= 0:
        return True


def initial_melee_attacks(attacker, defender):
    """This *should* just do the initial melee attacks a single time.
    Currently broken.
    """
    
    log = {
        'Total Damage': 0
    }
    
    for weapon, values in attacker.weapons.items():
        for attacks in range(values['rof']):
            result = melee_attack_roll(attacker, defender)
            
            if result == 'Hit!':
                damage = damage_roll(attacker, defender, values['p+s'])
                defender.hp = defender.hp - damage
                log['Total Damage'] += damage
            else:
                continue
            
            if is_dead(defender):
                return log
    return log


def rounds_to_kill(attacker, defender):
    """Returns the mean number of rounds it would take the attacker to kill the defender.
    Assumes the defender takes no actions.
    
    TODO: return a dictionary of lists
    """
    
    x = []
    
    for _ in range(num_of_tests):
        a = copy.copy(attacker)
        d = copy.copy(defender)
    
        log = {
            'Rounds': 0,
            'Total Damage': 0
        }
        
        # stop after the target is dead or 10 rounds have passed
        while not is_dead(d) and log['Rounds'] < 10:
            log['Rounds'] += 1
            log['Total Damage'] += initial_melee_attacks(a,d)['Total Damage']
            
        x.append(log['Rounds'])
    
    return st.mean(x)


def damage_per_round(attacker, defender):
    """text"""
    
    x = []
    
    for _ in range(num_of_tests):
        a = copy.copy(attacker)
        d = copy.copy(defender)
        
        x.append(initial_melee_attacks(a, d)['Total Damage'])
    
    return st.mean(x)

In [4]:
# research suites

def standard_output(a, d):
    output = {
        'Damage per round': damage_per_round(a,d),
        'Rounds to kill': rounds_to_kill(a,d)
    }
    
    return output

In [5]:
# workbench


In [8]:
# laboratory
num_of_tests = 10000

x = FeralWarpwolf(str_buff=0, mat_buff=0)
y = FeralWarpwolf(str_buff=0, mat_buff=0)

x.mind_disabled=False
x.body_disabled=False

xy = standard_output(x,y)
yx = standard_output(y,x)

print(xy)
print(yx)

{'Damage per round': 3.8961, 'Rounds to kill': 8.0805}
{'Damage per round': 9.8939, 'Rounds to kill': 3.6499}


In [7]:
# todo

## warbeast damage tracks (mind, body, spirit)
## chance to hit (suite: where to spend boost)
## new_unit class that populates from unit dicts
## if attack: or if hit: true instead of 'Hit!'
## crit interactions with boosted